In [7]:
import numpy as np

ntsteps = 3

# Load data
predict = np.load("predictions.npy")  # shape = (batch_no, loc/vel, dataset_in_batch, n_nodes, timesteps, 3_directions)
print(predict.shape)
nDataset = predict.shape[0]//ntsteps

data = np.zeros(shape=(nDataset,ntsteps,predict.shape[1],predict.shape[2]))
print(nDataset)

l = 0 
for i in range(ntsteps):
  data[i,:,:,:] = predict[l:l+3,:,:]
  l += ntsteps

data.shape

coords = data[:, :, :, 0:3]  # Extract coordinates
vel = data[:, :, :, 3:6]  # Extract velocities
connect = np.load("connectivity.npy")  # Load connectivity data

# Define the Tecplot file writing function
def write_tecplot_dat(filename, coords, vel, connectivity):
  n_nodes = coords.shape[1]
  n_elements = connectivity.shape[0]

  for t in range(ntsteps):
    with open(filename+'%i.dat'%t, 'w') as f:
      f.write('TITLE="3D TRIANGULAR SURFACE DATA"\n')
      f.write('VARIABLES="X","Y","Z"\n')
      f.write(f'ZONE T="unstruc" N={n_nodes} E={n_elements} F=FEPOINT ET=TRIANGLE\n')

      # Write node data
      for i in range(n_nodes):
        x, y, z = coords[t, i, :]
        u, v, w = vel[t, i, :]
        f.write(f'{x} {y} {z}\n')

      # Write connectivity data
      for conn in connectivity:
        f.write(f'{conn[0] + 1} {conn[1] + 1} {conn[2] + 1}\n')  # Tecplot uses 1-based indexing

# Write the data to a Tecplot .dat file
for t in range(nDataset):   
  write_tecplot_dat("./Post_Proc_Data/membrane/%i_"%t, coords[t,:,:,:], vel[t,:,:,:], connect)

(57, 1757, 6)
19
